In [ ]:
failuresPath = '/user/s292129/data/exam_01_2021/Failures.txt'
# schema : RID , FailTypeCode, Date, Time
robotsPath = '/user/s292129/data/exam_01_2021/Robots.txt'
# schema: RID, PlantID, IP
prodPlantsPath = '/user/s292129/data/exam_01_2021/ProductionPlants.txt'
# schema: PlantID, City, Country

In [ ]:
# Task 1: City with the maximum number of robots

prodPlantsRDD =  sc.textFile(prodPlantsPath).cache()

# We map each line of productionPlants.txt associated with italian cities in pairs (plantID, city)

prodPlant_city = prodPlantsRDD.filter(lambda line: line.split(',')[2] ==  'Italy' ).map(lambda line: ( line.split(',')[0], line.split(',')[1] ) )

robotsRDD = sc.textFile( robotsPath ).cache()

# We map each line of robots.txt in pairs (plantID, +1) and then we join this records with the ones of prodPlant_city in order to count the number of robots for each city

robots_oneRDD = robotsRDD.map( lambda line: ( line.split(',')[1], +1 ) )

pID_cityone = prodPlant_city.join( robots_oneRDD )

city_numRobots = pID_cityone.values().reduceByKey(lambda v1, v2: v1+v2).cache()

# We compute the maximum number of robot per city with a reduce action and then we select the cities associated with this maximum value obtained

maxRobotsPerCity = city_numRobots.reduce( lambda pair1, pair2: max( pair1[1], pair2[1] ) )

city_numRobots.filter( lambda pair: pair[1] == maxRobotsPerCity ).keys().collect()

In [ ]:
# TASK 2 : Maximum number of failures per robot for each prod plant 

failuresRDD = sc.textFile(failuresPath) 


In [ ]:

rid_one  = failuresRDD.map( lambda line:  ( line.split(',')[0], +1 ) )

rid_pid = robotsRDD.map( lambda line:  ( line.split(',')[0], line.split(',')[1] ) )


In [ ]:
# We join the pairs (rid, pid) with (rid, 1) to compute the number of failures for each distinct production plant and robot

rid_pidone = rid_pid.join(rid_one)

In [ ]:
pidrid_one = rid_pidone.map( lambda pair: ( ( pair[1][0], pair[0] ), +1 ) )

pidrid_numFailures = pidrid_one.reduceByKey( lambda v1, v2: v1+v2 )


In [ ]:
# We compute the maximum number of failures associated with a robot for each production plant with a reduceByKey transformation

pid_numFailures = pidrid_numFailures.map( lambda pair: ( pair[0][0], pair[1] ) ) 


pid_maxFailuresPerRobot = pid_numFailures.reduceByKey( lambda v1, v2: max(v1, v2) )

# We map all the lines of ProductionPlants.txt with a pair (plantid, 0) and then with subtractByKey we retrieve only the lines with a plantID that is not associated with a robot for which failures have occurred

pid_zeroRDD = prodPlantsRDD.map( lambda line: ( line.split(',')[0], 0) )

pid_zeroRDD = pid_zeroRDD.subtractByKey( pid_maxFailuresPerRobot )

# Finally we use union to get the finalRDD with the maximum number of failures for each production plant

finalRDD = pid_maxFailuresPerRobot.union( pid_zeroRDD )

In [ ]:
finalRDD.collect()